## Importar el archivo de tweets en CSV para el entrenamiento
Se importar el archivo y se eliminan las columnas que no se necesitan

Basado en: https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90
Indice: https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74

In [9]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re

#Definir el nombre de las columnas del archivo
cols = ['id','XX','Polaridad','Texto']

#Importar el archivo en memoria 
#Se indica que no tiene encabezados
#Se usan los nombres definidos anteriormente los nombres de las columnas
df = pd.read_csv("tweets.csv",header=None, names=cols)

#Se botan las columnas que no se necesitan
df.drop(['id','XX'],axis=1,inplace=True)

#Se agrega columna de longitud original para validaciones luego del pre proccesamiento
df['LongitudOriginal'] = [len(str(t)) for t in df.Texto]

#Imprimir las primeras líneas del archivo para validar una carga correcta
df.head()

,Polaridad,Texto,LongitudOriginal
0,NONE,Salgo de #VeoTV que día más largoooooo,40
1,NEU,@PauladeLasHeras No te libraras de ayudar meno...,63
2,P,@marodriguezb Gracias MAR,25
3,N+,Off pensando en el regalito Sinde la que se va...,128
4,P+,Conozco a alguien q es adicto al drama! Ja ja ...,65


In [10]:
#Imprimir el total de tweets por polaridad
df.Polaridad.value_counts()

P+      1654
NONE    1482
N       1334
P       1234
N+       846
NEU      669
Name: Polaridad, dtype: int64

In [24]:
df[1024:1026]

,Polaridad,Texto,LongitudOriginal
1024,P+,Todavía me estoy riendo con lo de la APP de ic...,125
1025,NEU,El Padre Ángel sigue el debate de #investidura...,91


In [31]:
patronURL = r'https?:/?/?[A-Za-z0-9./]+'
patronWWW = r'www.[^ ]+'

def tweet_cleaner(text):    
    #Reemplaza URLs con una "palabra" única
    stripped = re.sub(patronURL, 'xurlx', text)
    stripped = re.sub(patronWWW, 'xurlx', stripped)
    
    #Quita caracter identificar de UTF-8 BOM si existe
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
        
    #Reemplaza números con una "´palabra" única
    letters_only = re.sub("\d+", "xnumx", clean)
    
    #Pasa todo a minúsculas
    lower_case = letters_only.lower()
    
    return lower_case

#Prueba el pre procesamiento en un grupo pequeño de registros
testing = df.Texto[1024:1026]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

[u'todav\xeda me estoy riendo con lo de la app de icuenca leed la explicaci\xf3n de la app es genial no hace falta q la descargu\xe9is',
 u'el padre \xe1ngel sigue el debate de #investidura desde tribuna de invitados ayer no lo vimos']

In [32]:
print "Pre procesando tweets...\n"
clean_tweet_texts = []
for i in xrange(df.Texto.count()):
    if( (i+1)%1000 == 0 ):
        print "%d de %d tweets procesados" % ( i+1, df.Texto.count() )                                                                    
    clean_tweet_texts.append(tweet_cleaner(str(df['Texto'][i])))
print "Pre procesamiento completado"

Pre procesando tweets...

1000 de 7218 tweets procesados
2000 de 7218 tweets procesados
3000 de 7218 tweets procesados
4000 de 7218 tweets procesados
5000 de 7218 tweets procesados
6000 de 7218 tweets procesados
7000 de 7218 tweets procesados
Pre procesamiento completado


In [33]:
#Generar dataframe con tweets pre procesados
clean_df = pd.DataFrame(clean_tweet_texts,columns=['Texto'])
clean_df['Polaridad'] = df.Polaridad
clean_df.head()

,Texto,Polaridad
0,salgo de #veotv que día más largoooooo,NONE
1,@pauladelasheras no te libraras de ayudar meno...,NEU
2,@marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N+
4,conozco a alguien q es adicto al drama! ja ja ...,P+


In [35]:
clean_df.to_csv('tweets_limpios.csv',encoding='utf-8')
csv = 'tweets_limpios.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,Texto,Polaridad
0,salgo de #veotv que día más largoooooo,NONE
1,@pauladelasheras no te libraras de ayudar meno...,NEU
2,@marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N+
4,conozco a alguien q es adicto al drama! ja ja ...,P+


In [36]:
#Verificar si luego del pre procesamiento quedaron registros NULL 
#Hay que revisarlos contra los tweets originales y verificar si hay algún error
my_df[my_df.isnull().any(axis=1)]

,Texto,Polaridad
4640,NaN,NONE


In [37]:
#Luego de revisar los registros NULL, se detectó que el único caso era porque no tenía comentario
#Se van a desechar los registros NULL
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7217 entries, 0 to 7216
Data columns (total 2 columns):
Texto        7217 non-null object
Polaridad    7217 non-null object
dtypes: object(2)
memory usage: 112.8+ KB
